# Minimal Colab Wrapper

This notebook contains **only** the execution calls.
All business logic is in the GitHub repository under `src/`.

## Philosophy
- **Code in GitHub:** All logic, functions, classes in `src/`
- **Colab as executor:** Just call functions and save results
- **Reports for Claude:** Generate markdown reports for local review


## Configuration

In [ ]:
# === UPDATE THESE ===
GITHUB_REPO = "https://github.com/your-username/competition-name.git"
DRIVE_BASE = "/content/drive/MyDrive/kaggle/competition-name"
EXPERIMENT_NAME = "baseline_v1"

## Setup (Run Once)

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# Clone repo
!rm -rf /content/repo
!git clone {GITHUB_REPO} /content/repo

# Install dependencies
%cd /content/repo
!pip install -q -e .

print("✓ Setup complete")

## Update Code (Run when you push changes)

In [ ]:
# Pull latest changes from GitHub
%cd /content/repo
!git pull origin main

# Restart runtime if needed:
# Runtime > Restart runtime

print("✓ Code updated")

## Data Analysis

In [ ]:
# Import your modules (all logic is in src/)
from src.data_loader import load_competition_data
from src.eda import analyze_data, create_eda_report

# Load data
train_df, test_df = load_competition_data(f"{DRIVE_BASE}/data/raw")

# Analyze (this function does all the work)
analysis_results = analyze_data(train_df, test_df)

# Generate report for Claude to read
report_path = create_eda_report(
    analysis_results,
    output_dir=f"{DRIVE_BASE}/outputs/reports",
    plots_dir=f"{DRIVE_BASE}/outputs/plots"
)

print(f"✓ Analysis complete: {report_path}")
print("📄 Open this file in Claude Code to review")

## Feature Engineering

In [ ]:
from src.feature_engineering import create_features, save_processed_data

# Create features (logic in src/feature_engineering.py)
train_processed = create_features(train_df)
test_processed = create_features(test_df)

# Save to Drive
save_processed_data(
    train_processed,
    test_processed,
    output_dir=f"{DRIVE_BASE}/data/processed"
)

print(f"✓ Features created: {train_processed.shape}")

## Train Model

In [ ]:
from src.models import train_experiment

# All training logic is in src/models.py
results = train_experiment(
    experiment_name=EXPERIMENT_NAME,
    train_data=train_processed,
    model_type="lightgbm",
    params={
        "n_estimators": 1000,
        "learning_rate": 0.01,
        "device": "gpu"
    },
    output_dirs={
        "models": f"{DRIVE_BASE}/models",
        "reports": f"{DRIVE_BASE}/outputs/reports",
        "plots": f"{DRIVE_BASE}/outputs/plots"
    }
)

print(f"✓ Training complete")
print(f"  Val Score: {results['val_score']:.4f}")
print(f"  Report: {results['report_path']}")
print("📄 Open report in Claude Code to review")

## Generate Predictions

In [ ]:
from src.inference import generate_submission

# All prediction logic is in src/inference.py
submission_path = generate_submission(
    model_path=results['model_path'],
    test_data=test_processed,
    output_dir=f"{DRIVE_BASE}/submissions",
    experiment_name=EXPERIMENT_NAME
)

print(f"✓ Submission created: {submission_path}")

## Submit to Kaggle (Optional)

In [ ]:
# Setup Kaggle API (use Colab secrets)
from google.colab import userdata
import os

os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')

!kaggle competitions submit \
    -c competition-name \
    -f {submission_path} \
    -m "{EXPERIMENT_NAME}: Baseline with basic features"

print("✓ Submitted to Kaggle")

---

## Next Steps

1. **Local:** Open the generated report in Claude Code
2. **Ask Claude:** "Review the experiment report and suggest improvements"
3. **Update code:** Edit `src/` files locally with Claude's help
4. **Push to GitHub:** `git push origin main`
5. **Run again:** Re-run this notebook to test changes
